In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import timeit
import pandas as pd
import numpy as np 
from tqdm import tqdm, tqdm_notebook

# Authentication

In [2]:
cid ="70739307db3e428cbcff6639e00832cb" 
secret = "2b317a42cbbc4a1c833c0ec33d8f0614"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function for getting tracks from playlist

In [3]:
def get_tracks_by_playlist(id):
     try:
         df=pd.DataFrame()
         playlist_name=[]   
         data=sp.user_playlist('spotify',id)
         playlist_name.append(data['name'])
         track_list = [i['track']['id'] for i in data['tracks']['items']]
         for i in track_list:
            name,artist,temp,popularity,album=[],[],[],[],[]
            result=sp.track(i)
            name.append(result['name'])
            album.append(result['album']['name'])
            for a in result['artists']:
                artist.append(a['name'])
            temp.append(artist[:])
            popularity.append(result['popularity'])
            df=pd.concat([df,pd.DataFrame({'playlist_id':id,'playlist_name':playlist_name,'track_id':i,'track_name':name,'artist':temp,'album':album,'popularity':popularity})])
         df.reset_index(drop=True,inplace=True)  
         return df
     except:
         return df

# Function for song Features 

In [4]:
def get_song_features(ids,none_counter,list=None):
    '''
    Get song features for each song in a list
    '''
    for id_ in tqdm(ids):
        try:
            list.append(sp.audio_features(id_)[0])
        except:
            none_counter=none_counter+1

In [5]:
def get_playlist_tracks_with_features(playlist_id):
    playlist_track_df=pd.DataFrame() 
    try:
        playlist_track_df=get_tracks_by_playlist(playlist_id)
        feature_list=[]
        none_counter=0
        get_song_features(playlist_track_df.track_id.to_list(),none_counter,list=feature_list)
        df_audio_features = pd.DataFrame.from_dict(feature_list,orient='columns')
        df_audio_features.rename(columns={'id':'track_id'},inplace=True)
        result = pd.merge(playlist_track_df, df_audio_features, on='track_id')
        return result,none_counter
    except:
        return playlist_track_df,0

In [6]:
# get_playlist_tracks_with_features('37i9dQZF1DXaXB8fQg7xif')

In [7]:
# list=['37i9dQZF1DX3RCeShx2suK','37i9dQZF1DX3zdRCeShx2suK','37i9dQZF1DXdMbUSbTOEeW']

In [8]:
def get_dataset(playlist_id_list):
    result=pd.DataFrame()
    for id in tqdm(playlist_id_list):
        data,counter=get_playlist_tracks_with_features(id)
        result=pd.concat([result,data])
    return result,counter    

In [9]:
# get_dataset(list)

In [10]:
# get_playlist_tracks_with_features('37i9dQZF1DX3RCeShx2suK')

In [11]:
# df,c=get_dataset(list)

In [12]:
# df

In [13]:
playlist_df=pd.read_csv(r"Playlist_reduce.csv")

In [14]:
playlist_id_list=playlist_df.Playlist_id.to_list()
# playlist_id_list

In [15]:
final_dataset,counter=get_dataset(playlist_id_list)

 49%|████████████████████████████████████████                                          | 22/45 [00:04<00:04,  4.87it/s]


 91%|██████████████████████████████████████████████████████████████████████████▋       | 41/45 [00:08<00:00,  4.64it/s]


 16%|█████████████▎                                                                     | 8/50 [00:01<00:08,  4.74it/s]


 44%|████████████████████████████████████                                              | 22/50 [00:04<00:05,  4.69it/s]


 72%|███████████████████████████████████████████████████████████                       | 36/50 [00:07<00:03,  4.37it/s]


...

 94%|█████████████████████████████████████████████████████████████████████████████     | 47/50 [00:09<00:00,  5.05it/s]


 18%|██████████████▉                                                                    | 9/50 [00:01<00:08,  5.05it/s]


 46%|█████████████████████████████████████▋                                            | 23/50 [00:05<00:05,  4.73it/s]


 74%|████████████████████████████████████████████████████████████▋                     | 37/50 [00:08<00:02,  4.51it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 269/269 [1:26:38<00:00, 19.33s/it]


In [17]:
final_dataset

,playlist_id,playlist_name,track_id,track_name,artist,album,popularity,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,37i9dQZF1DX5NXVhu59GCz,Acid Blues,1I6l01BaBgBCVkfQCpIlR6,Truckin' - 2013 Remaster,[Grateful Dead],American Beauty (2013 Remaster),59,0.644,0.623,11,...,0.02020,0.0617,0.676,126.553,audio_features,spotify:track:1I6l01BaBgBCVkfQCpIlR6,https://api.spotify.com/v1/tracks/1I6l01BaBgBC...,https://api.spotify.com/v1/audio-analysis/1I6l...,306974,4
1,37i9dQZF1DX5NXVhu59GCz,Acid Blues,7uEw55wwByGZnEUTX0MaEI,Fox Is Sick,[The Savage Resurrection],The Savage Resurrection,26,0.687,0.580,7,...,0.19000,0.2060,0.964,129.415,audio_features,spotify:track:7uEw55wwByGZnEUTX0MaEI,https://api.spotify.com/v1/tracks/7uEw55wwByGZ...,https://api.spotify.com/v1/audio-analysis/7uEw...,162587,4
2,37i9dQZF1DX5NXVhu59GCz,Acid Blues,4GVRHAwABAV0v63YX9Go0p,Slip Inside This House - Stereo LP Version,[The 13th Floor Elevators],Easter Everywhere,44,0.555,0.694,9,...,0.00000,0.1230,0.602,127.423,audio_features,spotify:track:4GVRHAwABAV0v63YX9Go0p,https://api.spotify.com/v1/tracks/4GVRHAwABAV0...,https://api.spotify.com/v1/audio-analysis/4GVR...,482278,4
3,37i9dQZF1DX5NXVhu59GCz,Acid Blues,67kP4RjiGxSggSO5YniuLC,Crimson & Clover,[Aguaturbia],Psychedelic Drugstore - Remastered,30,0.272,0.832,0,...,0.14700,0.0547,0.625,85.399,audio_features,spotify:track:67kP4RjiGxSggSO5YniuLC,https://api.spotify.com/v1/tracks/67kP4RjiGxSg...,https://api.spotify.com/v1/audio-analysis/67kP...,635272,4
4,37i9dQZF1DX5NXVhu59GCz,Acid Blues,5FDGmW9TgycCTVwPnKZ4xC,Baby Please Don't Go,[The Amboy Dukes],Journey To The Center Of The Mind,34,0.404,0.612,0,...,0.19000,0.6410,0.663,93.851,audio_features,spotify:track:5FDGmW9TgycCTVwPnKZ4xC,https://api.spotify.com/v1/tracks/5FDGmW9TgycC...,https://api.spotify.com/v1/audio-analysis/5FDG...,338720,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,37i9dQZF1DX8jnAPF7Iiqp,Run 'N' Bass 170-175 BPM,43sYrQEm3Bjdt8qRpD9kx8,Big Sound,[Calyx & TeeBee],Big Sound,34,0.489,0.983,5,...,0.43200,0.1610,0.385,107.328,audio_features,spotify:track:43sYrQEm3Bjdt8qRpD9kx8,https://api.spotify.com/v1/tracks/43sYrQEm3Bjd...,https://api.spotify.com/v1/audio-analysis/43sY...,309070,5
46,37i9dQZF1DX8jnAPF7Iiqp,Run 'N' Bass 170-175 BPM,7wUE5I7YTCQBHNGyZQChzh,CTK4000,"[The Upbeats, Workforce, Tiki Taane]",CTK4000,35,0.671,0.903,11,...,0.53700,0.1860,0.145,172.044,audio_features,spotify:track:7wUE5I7YTCQBHNGyZQChzh,https://api.spotify.com/v1/tracks/7wUE5I7YTCQB...,https://api.spotify.com/v1/audio-analysis/7wUE...,302791,4
47,37i9dQZF1DX8jnAPF7Iiqp,Run 'N' Bass 170-175 BPM,2q7kfK2s7o9L6UXY857PDc,Levitate Your Mind,"[Degs, Unglued]",Levitate Your Mind,46,0.623,0.909,10,...,0.00405,0.0829,0.668,171.981,audio_features,spotify:track:2q7kfK2s7o9L6UXY857PDc,https://api.spotify.com/v1/tracks/2q7kfK2s7o9L...,https://api.spotify.com/v1/audio-analysis/2q7k...,251210,4
48,37i9dQZF1DX8jnAPF7Iiqp,Run 'N' Bass 170-175 BPM,0FajeedO3qoZja4lEWEzPM,Good to Me,[Friction],Good to Me,58,0.409,0.924,1,...,0.06000,0.0989,0.194,173.999,audio_features,spotify:track:0FajeedO3qoZja4lEWEzPM,https://api.spotify.com/v1/tracks/0FajeedO3qoZ...,https://api.spotify.com/v1/audio-analysis/0Faj...,181724,4


In [18]:
counter

0

In [19]:
final_dataset.to_csv('Final_dataset.csv',index=False)